In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
import matplotlib.pyplot as plt
import warnings

plt.style.use('seaborn')
#plt.style.use('seaborn-colorblind') #alternativa
#plt.rcParams['figure.figsize'] = [16,9]
plt.rcParams['figure.dpi'] = 300
warnings.simplefilter(action = 'ignore', category = FutureWarning)

# Dados Financeiros e Preprocessamento

## Coletando dados do Yahoo Finance

In [4]:
import pandas as pd
import yfinance as yf

In [23]:
df_yahoo = yf.download('AAPL', #podemos passar mais de um Ticker ['AAPL', 'MSFT']
                      start = '2000-01-01',
                      end = '2010-12-31',
                      #auto_adjust=True, #podemos usar para trazer o preço ajustado
                      # actions='inline', #podemos trazer Dividendos
                      progress = False)

In [24]:
print(f'Downloaded {df_yahoo.shape[0]} linhas de dados.')
df_yahoo.tail()

Downloaded 2766 linhas de dados.


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-12-23,11.607143,11.612500,11.541786,11.557143,9.938018,223157200
2010-12-27,11.530357,11.622857,11.482857,11.595714,9.971185,249816000
2010-12-28,11.639643,11.666429,11.609286,11.623929,9.995453,175924000
2010-12-29,11.650714,11.658929,11.610714,11.617500,9.989920,163139200
2010-12-30,11.624286,11.625357,11.537500,11.559286,9.939864,157494400


## Coletando dados do Quandl

Válido até dados de 2018, ie, sem atualização recente

In [26]:
import pandas as pd
import quandl

In [27]:
QUANDL_KEY = 'n82HGKMyq5xHHriyK6wE'
quandl.ApiConfig.api_key = QUANDL_KEY

In [35]:
df_quandl = quandl.get(dataset = 'WIKI/AAPL',
                      start_date = '2000-01-01',
                      end_date = '2010-12-31')

In [36]:
print(f'Downloaded {df_quandl.shape[0]} linhas de dados.')
df_quandl.head()

Downloaded 2767 linhas de dados.


,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2000-01-03,104.87,112.50,101.69,111.94,4783900.0,0.0,1.0,3.369314,3.614454,3.267146,3.596463,133949200.0
2000-01-04,108.25,110.62,101.19,102.50,4574800.0,0.0,1.0,3.477908,3.554053,3.251081,3.293170,128094400.0
2000-01-05,103.75,110.56,103.00,104.00,6949300.0,0.0,1.0,3.333330,3.552125,3.309234,3.341362,194580400.0
2000-01-06,106.12,107.00,95.00,95.00,6856900.0,0.0,1.0,3.409475,3.437748,3.052206,3.052206,191993200.0
2000-01-07,96.50,101.00,95.50,99.50,4113700.0,0.0,1.0,3.100399,3.244977,3.068270,3.196784,115183600.0


## Coletando dados do Intrinio

In [39]:
import intrinio_sdk
import pandas as pd

In [45]:
intrinio_sdk.ApiClient().configuration.api_key['api_key'] = 'OjJmODRlNWZiNjU0Nzg1OTAzNzdjNDJjMjI1OTZmMGVl'
security_api = intrinio_sdk.SecurityApi()

In [49]:
r = security_api.get_security_stock_prices(identifier='AAPL',
                                           start_date='2000-01-01',
                                           end_date='2021-12-31',
                                           frequency='daily',
                                           page_size=10000)

In [50]:
response_list = [x.to_dict() for x in r.stock_prices]
df_intrinio = pd.DataFrame(response_list).sort_values('date')
df_intrinio.set_index('date', inplace=True)

In [51]:
print(f'Downloaded {df_intrinio.shape[0]} linhas de dados.')
df_intrinio.tail()

Downloaded 5238 linhas de dados.


,intraperiod,frequency,open,high,low,close,volume,adj_open,adj_high,adj_low,adj_close,adj_volume,factor,split_ratio,dividend,change,percent_change,fifty_two_week_high,fifty_two_week_low
date,,,,,,,,,,,,,,,,,,,
2020-10-06,False,daily,115.70,116.12,112.2500,113.16,161498212.0,115.70,116.12,112.2500,113.16,161498212.0,1.0,1.0,0.0,-3.34,-0.0287,137.98,52.91
2020-10-07,False,daily,114.62,115.55,114.1300,115.08,96848985.0,114.62,115.55,114.1300,115.08,96848985.0,1.0,1.0,0.0,1.92,0.0170,137.98,52.91
2020-10-08,False,daily,116.25,116.40,114.5901,114.97,83477153.0,116.25,116.40,114.5901,114.97,83477153.0,1.0,1.0,0.0,-0.11,-0.0010,137.98,52.91
2020-10-09,False,daily,115.28,117.00,114.9200,116.97,100506865.0,115.28,117.00,114.9200,116.97,100506865.0,1.0,1.0,0.0,2.00,0.0174,137.98,52.91
2020-10-12,False,daily,120.06,125.18,119.2845,124.40,237940124.0,120.06,125.18,119.2845,124.40,237940124.0,1.0,1.0,0.0,7.43,0.0635,137.98,52.91


## Convertando preço para retorno

Os preços de ações não são estacionários, ie, suas propriedades estatísticas como média e variância, mudam com o tempo, em que podemos observar também sazonalidade e tendência. Ao transformar preços em retornos, tornamos a série temporal estacionária, para que possamos modelar mais facilmente.

Trabalhamos com dois retornos:
    
***
$\mathbf{\text{Retorno simples}}$<br>

   Eles agregam sobre ações. O retorno simples de uma carteira é a média ponderada da soma do retorno de cada ativo do portifpolio individualizado
   
   
   $$R_{t} = \frac{(P_{t} - P_{t-1})}{ P_{t-1}} = \frac{P_{t}}{ P_{t-1}}-1$$
   
  
   
***
$\mathbf{\text{Log-retorno}}$<br>

   Eles agregam sobre o tempo. O log-retorno para um dado mês é a soma dos log-retornos diários desse mês
   
   $$r_{t} = log\displaystyle \Bigg(\frac{P_{t}}{ P_{t-1}}\Bigg) = log(P_{t}) -  log(P_{t-1})$$


***
A melhor prática ao trabalhar com preços de ações é usar valores ajustados, pois eles contabilizam possíveis ações corporativas, como desdobramentos de ações

In [52]:
import pandas as pd
import numpy as np
import yfinance as yf

Fazemos o download da base e mantemos o preço de fechamento ajustado apenas

In [72]:
df = yf.download('AAPL',
                start = '2000-01-01',
                end = '2010-12-31',
                progress = False)
df = df.loc[:, ['Adj Close']]
df.rename(columns = {'Adj Close': 'adj_close'}, inplace = True)

df.head()

,adj_close
Date,
2000-01-03,0.859423
2000-01-04,0.786965
2000-01-05,0.798481
2000-01-06,0.729382
2000-01-07,0.763932


Convertemos o preço de fechamento em retorno simples e log retorno.

In [73]:
df['simple_rtn'] = df.adj_close.pct_change() #conseguimos definir o número de lags, mas nesse caso é 1 default
df['log_rtn'] = np.log(df.adj_close/df.adj_close.shift(1))

df.head()

,adj_close,simple_rtn,log_rtn
Date,,,
2000-01-03,0.859423,NaN,NaN
2000-01-04,0.786965,-0.084310,-0.088077
2000-01-05,0.798481,0.014633,0.014527
2000-01-06,0.729382,-0.086538,-0.090514
2000-01-07,0.763932,0.047369,0.046281


## Incluindo inflação

Para incluir a inflação nos retornos de preço do período estudado, usamos o indicador CPI(Consumer Price Index) da biblioteca Quandl e calculamos o pct_change(retono simples) do index e podemos juntar os dados de inflação com os retornos das ações da Apple pela fórmula

$$R^{r}_{t} = \displaystyle \Bigg(\frac{1+R_{t}}{1+ \pi_{t}}\Bigg) - 1$$

Onde $\pi_{t}$ é a taxa de inflação

In [74]:
df_all_dates = pd.DataFrame(index = pd.date_range(start = '2000-01-01',
                                                 end = '2010-12-31'))
df = df_all_dates.join(df[['adj_close']], how='left') \
 .fillna(method='ffill') \
 .asfreq('M')

# join confere a junção entre bases modo "left"
# Em casos em que o último dia do mês não é um trading day, pegamos a última referencia pelo ffill
# Selecionando a frequência 'M', pegamos apenas a última data de cada mês

df.head()

,adj_close
2000-01-31,0.796561
2000-02-29,0.880057
2000-03-31,1.042729
2000-04-30,0.952515
2000-05-31,0.644927


Download dos dados de inflação pelo Quandl

In [79]:
df_cpi = quandl.get(dataset='RATEINF/CPI_USA',
                    start_date='1999-12-01',
                    end_date='2010-12-31')
df_cpi.rename(columns={'Value':'cpi'}, inplace = True)
df_cpi.head()

,cpi
Date,
1999-12-31,168.3
2000-01-31,168.8
2000-02-29,169.8
2000-03-31,171.2
2000-04-30,171.3


Merge dos dados de inflação com os preços

In [80]:
df_merged = df.join(df_cpi, how = 'left')
df_merged.head()

,adj_close,cpi
2000-01-31,0.796561,168.8
2000-02-29,0.880057,169.8
2000-03-31,1.042729,171.2
2000-04-30,0.952515,171.3
2000-05-31,0.644927,171.5


Cálculo do retorno simples e taxa de inflação

In [81]:
df_merged['simple_rtn'] = df_merged.adj_close.pct_change()
df_merged['inflation_rate'] = df_merged.cpi.pct_change()
df_merged.head()

,adj_close,cpi,simple_rtn,inflation_rate
2000-01-31,0.796561,168.8,NaN,NaN
2000-02-29,0.880057,169.8,0.104820,0.005924
2000-03-31,1.042729,171.2,0.184842,0.008245
2000-04-30,0.952515,171.3,-0.086517,0.000584
2000-05-31,0.644927,171.5,-0.322922,0.001168


Ajustando os retornos pela infação

In [82]:
df_merged['real_rtn'] = (1 + df_merged['simple_rtn'])/(1+df_merged['inflation_rate']) - 1
df_merged.head()

,adj_close,cpi,simple_rtn,inflation_rate,real_rtn
2000-01-31,0.796561,168.8,NaN,NaN,NaN
2000-02-29,0.880057,169.8,0.104820,0.005924,0.098314
2000-03-31,1.042729,171.2,0.184842,0.008245,0.175153
2000-04-30,0.952515,171.3,-0.086517,0.000584,-0.087050
2000-05-31,0.644927,171.5,-0.322922,0.001168,-0.323711


## Mudando a frequência

A regra de bolso para se mudar a frequência temporal pode ser:

 - Multiplicar ou dividir os log-retornos pelo total de períodos de tempo;
 - Multiplicar a volatilidade pela raíz quadrada pelo número de períodos temporais;
 
A volatilidade realizada é dada pela fórmula:

$$RV = \displaystyle \sqrt {\sum \limits _{i =1} ^T {r _t ^2}} $$
    
Abaixo calculamos as volatilidades mensais realizadas para a Apple usando retornos diários e então anualizamos esses valores. A volatilidade realizada é geralmente usada para volatilidades diárias usando retornos intra-diários. Os passos que seguimos nesse caso são:

 - Download dos dados e cálculo dos log-retornos;
 - Cálculo da volatilidade realizada ao longo dos meses;
 - Anualizar os valores pela multiplicação por $\sqrt {12} $, já que partimos de valores mensais.

In [84]:
import pandas as pd
import yfinance as yf

# Download dos dados
df = yf.download('AAPL',
                start = '2000-01-01',
                end = '2010-12-31',
                auto_adjust = False,
                progress = False)

# Mantemos apenas o preço ajustado de fechamento
df = df.loc[:, ['Adj Close']]
df.rename(columns = {'Adj Close': 'adj_close'}, inplace = True)

# Calculamos os log-retornos
df['log_rtn'] = np.log(df.adj_close/df.adj_close.shift(1))

# Removemos dados redundantes
df.drop('adj_close', axis = 1, inplace = True)
df.dropna(axis = 0, inplace = True)

df.head()

,log_rtn
Date,
2000-01-04,-0.088077
2000-01-05,0.014527
2000-01-06,-0.090514
2000-01-07,0.046281
2000-01-10,-0.017744
